In [ ]:
def PrepData_for_RAG():
  # RAG dataset
  # In one time, ~2500 rows only were being returned by PubMed API
  PubMed_articles = pd.read_csv("/content/sample_data/PubMed_articlesV2.csv", names=["article_abstract"])

  # debugging, Please ignore.
  print(PubMed_articles.head(3))

  rag_docs = []
  for index, row in PubMed_articles.iterrows():
    rag_docs.append(row["article_abstract"])

  # debugging, Please ignore.
  print(rag_docs)

  rag_docsV2 = []
  for each_str in rag_docs:
    if isinstance(each_str, str):
      rag_docsV2.append(each_str)

  doc_embeddings = embedding_model.encode(rag_docsV2)
  ids = [str(i) for i in range(len(rag_docsV2))]

  # Add embeddings to Vector DB
  # 1) Create a client that will persist to "./chroma_db"
  client = chromadb.Client(Settings(
      persist_directory="/chroma_db",
      is_persistent=True
  ))

  # 2) Create (or get) your collection
  collection = client.get_or_create_collection(name="beautyRAGtool")

  collection.add(
      ids=ids,           # e.g. ["doc1", "doc2", …]
      documents=rag_docsV2,    # e.g. ["Text of doc1", "Text of doc2", …]
      embeddings=doc_embeddings,   # e.g. [[0.1, 0.2,…], [0.3,0.4,…], …]
  )

  # Test with a sample query
  collection = client.get_collection("beautyRAGtool")
  return collection